In [ ]:
import json

import polars as pl

from src.lib import version_extractor, name_extractor

In [ ]:
df = pl.read_json('../../data/big_query/bq_results.json', json_lines=True)

In [ ]:
df_sorted = df.sort(['name', 'version', 'upload_time'], reverse=[False, True, False])
print(df_sorted)

In [ ]:
# Eliminate duplicates
df_unique = df_sorted.unique(subset=['name', 'version'])
print(df_unique)

In [ ]:
# Used to find the true number of packages
print(len(df_unique.groupby('name').groups()))

In [ ]:
from datetime import datetime


def convert_to_rfc3339(date: str):
    try:
        parsed_date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f %Z").astimezone()
    except ValueError:
        parsed_date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S %Z").astimezone()
    return str(parsed_date.isoformat())

In [ ]:
df_normalized_time = df_unique.with_columns([
    pl.col('upload_time').apply(convert_to_rfc3339)
])
print(df_normalized_time)

In [ ]:
data = json.loads(df_normalized_time.to_pandas().to_json(orient='records'))

In [ ]:
def extract_name_and_version(dependency_version_string: str) -> (str, str):
    if dependency_version_string.find("extra") != -1:
        return None, None
    dep_name = name_extractor(dependency_version_string)
    dep_version = version_extractor(dependency_version_string)
    if dep_name and version:
        return dep_name, dep_version

results: dict[str, dict[str, dict]] = {}

for index, dictionary in enumerate(data):
    name = dictionary['name']
    version = dictionary['version']
    upload_time = dictionary['upload_time']

    if name not in results:
        normalized_form = {
            'name': name,
            'versions': {}
        }
    else:
        normalized_form = results.get(name)

    normalized_form['versions'][version] = {
        'timestamp': upload_time,
        'dependencies': {}
    }

    for dep in dictionary['requires_dist']:

        dependency_name, dependency_version = extract_name_and_version(dep)
        # Only add dependency if we successfully extracted its name and version
        if dependency_name is not None and dependency_version is not None:
            normalized_form['versions'][version]['dependencies'][dependency_name] = dependency_version
    results[name] = normalized_form

del data
# results

In [ ]:
final_result = {'pkgs': list(results.values())}
# Uncomment next line to use the old format for the JSON file
# final_result = list(results.values())

with open('../../data/output/pypi-bq-dependencies420k-no-extra-correct.json', 'w') as file:
    json.dump(final_result, file)